In [135]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

driver = webdriver.Chrome()
url = 'https://www.google.co.kr/maps/place/%EA%B0%95%EC%9B%90%ED%8A%B9%EB%B3%84%EC%9E%90%EC%B9%98%EB%8F%84+%EA%B0%95%EB%A6%89%EC%8B%9C/data=!3m1!4b1!4m6!3m5!1s0x3561e5e1954f0f5b:0x80b7c6519c3d1d46!8m2!3d37.7563022!4d128.8990843!16zL20vMDF0a3lj?hl=ko&entry=ttu&g_ep=EgoyMDI0MTAyOS4wIKXMDSoASAFQAw%3D%3D'
driver.get(url)

time.sleep(4)

hotel_click = '''
#assistive-chips > div > div > div > div.e07Vkf.kA9KIf > div > div > div > div > div.rHNip.cRLbXd > div.dryRY > div:nth-child(2) > button > div > div.L6Bbsd
'''

time.sleep(4)

driver.find_element(By.CSS_SELECTOR, hotel_click).click() 

time.sleep(4)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

time.sleep(4)

hotel_links = soup.select('a.hfpxzc')

time.sleep(2)

# 각 링크에서 href를 추출하여 Selenium으로 이동
list = []

for link in hotel_links:
    hotel_url = link.get('href')  # 링크의 href 속성 추출
    driver.get(hotel_url)  # Selenium을 사용해 해당 URL로 이동
    hotel = link.get('aria-label')
    time.sleep(3)  # 페이지 로딩 대기
    # 이제 리뷰 버튼 누를 차례
    driver.find_elements(By.CSS_SELECTOR, 'button.hh2c6[data-tab-index="2"]')[0].click()
    # 그다음에 각각의 리뷰에 접근할 차례
    time.sleep(3)
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    review_links = soup2.select('div.jftiEf')
    for review in review_links:
        star = review.find('span', class_='fzvQIb')
        if star:
            match = re.search(r'\d+/\d+', star.text)
            if match:
                result = match.group(0)
                star = result 
        text = review.find('span', class_='wiI7pd').get_text()
        date = review.find('span', class_='xRkPPb').text.split()[0]
        list.append([hotel, star, date, text])
df = pd.DataFrame(list, columns = ['hotel_name', 'star', 'date', 'text'])
df

    

,hotel_name,star,date,text
0,스카이베이호텔 경포,3/5,1주,"룸컨디션 좋고, 오션뷰도 아주 좋았습니다. 해변가 가까워 문을 열어두니 파도소리가 ..."
1,스카이베이호텔 경포,5/5,1달,추석연휴에 가족들과 하루 이용했습니다. 호텔 바로 앞이 바닷가라 늦은 밤까지 바다 ...
2,스카이베이호텔 경포,4/5,2달,로비 카운터 직원 교육이 필요한 호텔입니다.\n기다리는 줄의 안내도 없었는데 줄을 ...
3,스카이베이호텔 경포,3/5,3주,비성수기 기준 가격대비 위치도 좋았고 깔끔한편입니다.\n다만 방음이 너무 안됩니다....
4,스카이베이호텔 경포,4/5,2달,지은지 얼마 안돼서 깨끗하고 위치는 최적입니다.|주변 맛집많고 바닷가 바로 코앞이라...
...,...,...,...,...
65,호텔 탑스텐,5/5,2달,"방도 매우 넓고 (주니어 스위트 기준 약28마넌) 쾌적합니다.\n숙소내부에 대게, ..."
66,호텔 탑스텐,5/5,2달,커플 쥬니어스위트.4박 청결하고 넓은룸 분위기좋음.\n석식 2번 먹엇는데 예전에비해...
67,호텔 탑스텐,5/5,1주,"바다뷰와 청결도가 최고입니다~\n온천도 피로회복 끝내주고요!\n강추,재방문의사100%"
68,호텔 탑스텐,1/5,3달,최악!\n15만원 내고 지저분한 옥탑방 체험하실 분은 예약하십시오.…


In [133]:
df.to_excel('GoogleReview.xlsx', index = False)